In [24]:
# 导入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [25]:
# 读取数据
!ls data/

df_train = pd.read_csv("data\\first_round_training_data.csv")

# 去掉多余的特征
df_train = df_train.drop(df_train.columns[[10,11,12,13,14,15,16,17,18,19,]], axis=1, )

# 特征10 换位到最后一列
df_test = pd.read_csv("data\\first_round_testing_data.csv")
tmp = df_test["Parameter10"].copy()
df_test.drop("Parameter10",axis=1, inplace=True)
df_test["Parameter10"] = tmp

a.py
first_round_testing_data.csv
first_round_training_data.csv
lgb_fisrt.csv
lgb1300round.csv


In [26]:
df_train.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Quality_label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,Pass
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,Fail
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,Fail
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,Fail
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,Fail


In [27]:
# 制作标签
def makeLabel(row):
    #print(row)
    if row["Quality_label"] == "Fail":
        return 0
    elif row["Quality_label"] == "Pass":
        return 1
    elif row["Quality_label"] == "Good":
        return 2
    elif row["Quality_label"] == "Excellent":
        return 3

df_train["Quality_label"] = df_train.apply(makeLabel, axis=1)   
df_train.head()


,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Quality_label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,1
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,0
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,0
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,0
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,0


In [28]:
def score(res, target):
    if len(res) != len(target):
        raise Exception("数据错误")
    
    trueNum = 0
    for i in range(len(res)):
        if res[i] == y_train.iloc[i]:
            trueNum += 1
    print(float(trueNum/len(res)))

In [96]:
# 划分数据集
df_target = df_train["Quality_label"]
train = df_train.drop(["Quality_label"], axis=1)


X_train,X_test, y_train, y_test = train_test_split(train,df_target,test_size=0.3, random_state=0)


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#对数据的训练集进行标准化

# x = X_train.copy()
# ss = StandardScaler()
# X= ss.fit_transform(x)


## model -1 逻辑回归
# clf = LogisticRegression(random_state=2, solver='lbfgs',multi_class='multinomial',max_iter = 1000).fit(X, y_train)
# print(clf.score(X,y_train))


### model -2  决策树 

# from sklearn.tree import DecisionTreeClassifier

# treeClf = DecisionTreeClassifier(
#                                      max_depth=7
#                                    # ,splitter = "random"
#                                     ,min_samples_split = 40
#                                     ,random_state = 6
    
#                                         )
# treeClf = treeClf.fit(X_train, y_train)
# res = treeClf.score(X_test,y_test)
# print(res)

### model-3  随机森林  # 最好成绩 0.44
# from sklearn.ensemble import RandomForestClassifier

# rfc = RandomForestClassifier(
#                           n_estimators=100  # 默认10，决策树的数量, 越多越好，从100开始。
                          
#                         , bootstrap=True   # 有放回采样
#                         , oob_score=False  # out-of-bag 使用没有抽到的数据做验证
#                         , n_jobs=None      #并行计算
#                         , verbose=0        # 打印训练日志
#                         , warm_start=False  # 热启动
                        
#                         #与决策树相关的参数
#                         , criterion='gini'  # 或 "entropy" 
#                         , random_state=None
#                         , class_weight=None
#                         , max_depth=None
#                         , min_samples_split=2
#                         , min_samples_leaf=1
#                         , min_weight_fraction_leaf=0.0
#                         , max_features='auto' # auto=特征数的开根号，log2=log2(n_features)， None = n_features
#                         , max_leaf_nodes=None
#                         , min_impurity_decrease=0.0
#                         , min_impurity_split=None
#                         )

# rfc = rfc.fit(X_train, y_train)
# score = rfc.score(X_test,y_test)
# print(score)


### model-4 xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)


model = xgb.XGBClassifier(max_depth=3, n_estimators=200, learn_rate=0.01)
model.fit(X_train, y_train)  
test_score = model.score(X_test, y_test)
print('test_score: {0}'.format(test_score))


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


test_score: 0.49944444444444447


In [98]:
dtrain

AttributeError: 'DMatrix' object has no attribute 'shape'

In [ ]:
### 调参过程

# score_list = []
# for c in range(340,380,2):
#     model = xgb.XGBClassifier(
#           max_depth=5
#         , learning_rate=0.1
#         , n_estimators=c   # step1: 360
#         , verbosity=1
#         , silent=None
#         , objective='reg:squarederror'
#         , booster='gbtree'
#         , min_child_weight=1

#         , scale_pos_weight=1
#         , base_score=0.5
#         , random_state=0
#         , seed=None
#         , missing=None
#         , importance_type='gain',
        
#         )
#     model.fit(X_train, y_train)  
#     test_score = model.score(X_test, y_test)
#     #print('test_score: {0}'.format(test_score))
#     score_list.append(test_score)
# print("best n_estimators: ", range(340,380,2)[score_list.index(max(score_list))])
# print("best score:", max(score_list))
# plt.figure(figsize=[8,4])
# plt.plot(range(340,380,2),score_list)

from sklearn.model_selection import GridSearchCV

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test,label=y_test)
cv_params = {'random_state': [42,2,3,2019,2000,10,1,6666]}
other_params = {    
                   # step1: 24
                    "n_estimators":5000
    
                  # step2: 4, 3
                    , "max_depth":4
                    , "min_child_weight" : 3
    
                  # step3:
                    , "subsample" : 0.7
                    , "colsample_bytree" : 1
    
                
                    , "learning_rate":0.1
                
                    , "verbosity":1
                    , "silent":None
                    , "objective":'binary:logistic'
                    , "booster": "gbtree"
                    , "n_jobs" : 4
                    , "nthread" : None
                    , "gamma" : 0
                  
                    , "max_delta_step" : 0

                    , "colsample_bylevel" : 1
                    , "colsample_bynode" : 1
                    , "reg_alpha" : 0
                    , "reg_lambda" : 1
                    , "scale_pos_weight" : 1
                    , "base_score" : 0.5
                  #  , "random_state" : 42
                    , "seed"  : None
                    , "missing" : None
                    , "importance_type" :'gain'
}

model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train, df_target)

evalute_result = optimized_GBM.cv_results_['mean_test_score']
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [87]:
#
model = xgb.XGBClassifier(learning_rate=0.1, n_estimators=24, max_depth=4, min_child_weight=3, seed=0,
                         subsample=0.7, colsample_bytree=1, gamma=0, reg_alpha=0, reg_lambda=1)
model.fit(train, df_target)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=24, n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=None, subsample=0.7, verbosity=1)

In [128]:
# 测试集
testRes = model.predict(df_test.iloc[:,1:-1])

testpredict = model.predict_proba(df_test.iloc[:,1:-1])


In [131]:
train.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385


In [124]:
y_pred = pd.DataFrame(
    testpredict, columns=["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]
)
y_pred["Group"] = df_test["Group"]

group_pred = y_pred.groupby("Group").mean()
for c in ["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]:
    group_pred[c] /= group_pred.sum(axis=1)
    

group_pred.to_csv('./result1.csv', index=True, header=True)
group_pred.head()

,Excellent ratio,Good ratio,Pass ratio,Fail ratio
Group,,,,
0,0.189286,0.255415,0.347859,0.207439
1,0.171206,0.385741,0.244959,0.198095
2,0.178032,0.300359,0.283827,0.237781
3,0.116606,0.365288,0.260838,0.257268
4,0.203831,0.264917,0.358745,0.172507


In [126]:
y_pred.head()

,Excellent ratio,Good ratio,Pass ratio,Fail ratio,Group
0,0.076295,0.207588,0.253257,0.462860,0
1,0.593125,0.144209,0.211633,0.051034,0
2,0.396704,0.172881,0.381864,0.048551,0
3,0.312404,0.150442,0.489301,0.047852,0
4,0.242988,0.185858,0.481353,0.089801,0


,Excellent ratio,Good ratio,Pass ratio,Fail ratio,Group
Group,,,,,
0,0.189286,0.255415,0.347859,0.207439,0
1,0.171206,0.385741,0.244959,0.198095,0
2,0.178032,0.300359,0.283827,0.237781,0
3,0.116606,0.365288,0.260838,0.257268,0
4,0.203831,0.264917,0.358745,0.172507,0
5,0.177974,0.381374,0.244866,0.195786,0
6,0.179501,0.355975,0.267049,0.197474,0
7,0.156005,0.435018,0.230203,0.178774,0
8,0.085940,0.648989,0.129771,0.135299,0


In [89]:
# 类别结果加到原始的测试集
df_test = pd.concat([df_test, pd.DataFrame(data=testRes)], axis=1,)

In [90]:
df_test.head()

,Group,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,0
0,0,0.235279,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852,0.195680,3
1,0,0.043799,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257,0.523976,0
2,0,21.466737,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257,0.523976,0
3,0,0.703795,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850,0.010192,2
4,0,10.447960,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850,0.010192,2


In [91]:
submit_csv = pd.read_csv("submit_example.csv")
submit_csv.head()

,Group,Excellent ratio,Good ratio,Pass ratio,Fail ratio
0,0,0.4,0.2,0.2,0.2
1,1,0.4,0.2,0.2,0.2
2,2,0.4,0.2,0.2,0.2
3,3,0.4,0.2,0.2,0.2
4,4,0.4,0.2,0.2,0.2


In [132]:
# 分组计算 概率
g = df_test.groupby("Group")
group_list = df_test.Group.unique()

for i in group_list:
    tmp = g.get_group(i)
    Excellent = len(tmp[tmp[0]==3])/50.0
    Good = len(tmp[tmp[0]==2])/50.0
    Pass = len(tmp[tmp[0]==1])/50.0
    Fail = len(tmp[tmp[0]==0])/50.0
    submit_csv[submit_csv["Group"]==i] = [i, Excellent, Good, Pass, Fail]

In [135]:
tmp = g.get_group(1)
tmp

,Group,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,0
50,1,0.162531,1759.813981,7.384585,0.000239,0.142638,0.001623,0.152059,0.035407,0.593081,1.403055,0
51,1,6.904254,32.273783,0.047780,0.000543,1.600654,1.495371,0.600827,17.850021,6.783967,0.073078,2
52,1,0.139577,16.387509,21.809453,0.006026,1.786601,1.230409,0.600827,17.850021,0.051850,0.027291,2
53,1,0.034670,0.233238,29.484492,3.595958,0.000338,0.000744,146.453658,0.035407,0.593081,26.938018,2
54,1,0.004270,0.691408,4088.082718,121.485426,1.786601,2.208755,0.152059,0.035407,0.593081,517.197610,0
55,1,0.000029,1.302675,0.002824,1.496771,17.962176,8.650538,0.152059,0.035407,0.593081,37.407054,1
56,1,211.330002,0.046003,0.093137,582256.661411,1.994149,3.965023,0.600827,2.931083,262.444835,0.377332,0
57,1,962.444694,0.077945,1.371315,25968.328954,1.600654,1.817391,0.038483,17.850021,0.004533,0.005285,2
58,1,0.000003,0.020378,4.537344,0.000185,3.095123,2.208755,0.038483,2.931083,2.005852,0.073078,2
59,1,556.133706,2.848193,0.025717,0.030247,2.772987,3.262468,2.374035,14.603595,0.593081,0.037897,1


In [93]:
submit_csv.head()

,Group,Excellent ratio,Good ratio,Pass ratio,Fail ratio
0,0,0.16,0.56,0.16,0.12
1,1,0.06,0.30,0.46,0.18
2,2,0.32,0.30,0.20,0.18
3,3,0.22,0.40,0.34,0.04
4,4,0.04,0.64,0.18,0.14


In [95]:

submit_csv.to_csv('./result.csv', index=False, header=True)